In [1]:
import xarray as xr
import matplotlib.pyplot as plt
from cmocean import cm # for oceanography-specific colormaps
from scipy.io import loadmat
import numpy as np
import re
from xml.etree import ElementTree as ET
from matplotlib.path import Path
import pandas as pd
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import math
import xoak
from tqdm import tqdm

In [2]:
### to do
def interp_JRA(
    path1 = None ,
    mesh_file = None,
    u_file =  None,
    v_file = None,
):
    #mesh
    ds = xr.open_dataset(mesh_file, chunks = {'time':1})
    #mask
    mask=ds.mask_rho.compute()
    mask = np.where(mask == 0, np.nan, mask)
    #lon and lat
    lon=ds.lon_rho.compute()
    lat=ds.lat_rho.compute()
    lons=(math.floor(lon.min().item())+360,math.floor(lon.max().values.item()+360))
    lats=(math.floor(lat.max().item()),math.floor(lat.min().item()))

    #import the JRA data
    # JRApath='/gxfs_work/geomar/smomw662/NHCS/Winds_input/JRA/'
    ds_u = xr.load_dataset(path1+u_file, engine="cfgrib").u10.sel(latitude=slice(*lats), longitude=slice(*lons))
    ds_v = xr.load_dataset(path1+v_file, engine="cfgrib").v10.sel(latitude=slice(*lats), longitude=slice(*lons))
    
    ds_u['longitude']=ds_u.longitude -360
    ds_v['longitude']=ds_v.longitude -360

    #timestep
    tsp = ds_u.time.shape[0]
    # Create the target fine grid (lon=602, lat=542)
    lon_fine = np.linspace(ds_u.longitude.values.min(), ds_u.longitude.values.max(), 602)
    lat_fine = np.linspace(ds_u.latitude.values.min(), ds_u.latitude.values.max(), 542)
    lon_grid_fine, lat_grid_fine = np.meshgrid(lon_fine, lat_fine)

    # Extract the original coarse grid (lon=40, lat=35)
    lon_coarse = ds_u.longitude.values.flatten()
    lat_coarse = ds_u.latitude.values.flatten()
    lon_grid_coarse, lat_grid_coarse = np.meshgrid(lon_coarse, lat_coarse)
    
    # Step 3: Initialize an empty array to store the interpolated data
    u_interp = np.zeros((tsp, 542, 602))
    v_interp = np.zeros((tsp, 542, 602))

    # Step 4: Loop over each time step
    for t in tqdm(range(tsp)):
        # Extract the data for the current time step
        u = ds_u.isel(time=t).values  # Assuming the variable is named 'Uwind'
        v = ds_v.isel(time=t).values  # Assuming the variable is named 'Uwind'
        
        # Flatten the coarse grid coordinates and data for interpolation
        coords_coarse = np.column_stack((lon_grid_coarse.flatten(), lat_grid_coarse.flatten()))
        uflat = u.flatten()
        vflat = v.flatten()
        
        # Flatten the target fine grid coordinates
        coords_fine = np.column_stack((lon_grid_fine.flatten(), lat_grid_fine.flatten()))
        
        # Cubic interpolation of U
        data_u = griddata(
            coords_coarse,  # Original coordinates (shape: (40*35, 2))
            uflat,  # Original data (shape: (40*35,))
            coords_fine,  # Target coordinates (shape: (602*542, 2))
            method='cubic'  # or 'linear' for faster results
        )

        # Perform interpolation (linear or cubic)
        data_v = griddata(
            coords_coarse,  # Original coordinates (shape: (40*35, 2))
            vflat,  # Original data (shape: (40*35,))
            coords_fine,  # Target coordinates (shape: (602*542, 2))
            method='cubic'  # or 'linear' for faster results
        )
        
        # Reshape the interpolated data to the fine grid shape
        u_interp[t, :, :] = data_u.reshape((542, 602)) *mask
        v_interp[t, :, :] = data_v.reshape((542, 602)) *mask

    # U interpolated data
    u10 = xr.DataArray(
        u_interp,
        dims=('time', 'lat', 'lon'),
        coords={
            'time': ds_u.time.values,
            'lat': lat_fine,
            'lon': lon_fine
        }
    )

    # V interpolated data
    v10 = xr.DataArray(
        v_interp,
        dims=('time', 'lat', 'lon'),
        coords={
            'time': ds_u.time.values,
            'lat': lat_fine,
            'lon': lon_fine
        }
    )

    u10 = u10.rename('u10')
    v10 = v10.rename('u10')

    return u10, v10

In [3]:
# Define the paths to your FESOM data files
path1 = "/gxfs_work/geomar/smomw662/NHCS/Winds_input/JRA/"  
mesh_file = "/gxfs_work/geomar/smomw662/NHCS/hincast_1980-2015/croco_avg_Y1980M01.nc"  # The FESOM mesh file

for year in tqdm(range(1980, 1980+1,1)):
    
    u_file = f"anl_surf.033_ugrd.reg_tl319.{year}010100_{year}123118"  # File containing U velocity
    v_file = f"anl_surf.034_vgrd.reg_tl319.{year}010100_{year}123118"  # File containing V velocity
    
    u_interp, v_interp =  interp_JRA(path1 = path1,
                                                 mesh_file = mesh_file, 
                                                 u_file = u_file,
                                                 v_file = v_file,
                                                )
    
    u_interp.drop_encoding().to_netcdf(f'/gxfs_work/geomar/smomw662/NHCS/Winds_input/JRA_interp/u10.{year}.nc') 
    v_interp.drop_encoding().to_netcdf(f'/gxfs_work/geomar/smomw662/NHCS/Winds_input/JRA_interp/v10.{year}.nc')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1464 [00:00<?, ?it/s]

  0%|          | 1/1464 [00:00<18:34,  1.31it/s]

  0%|          | 2/1464 [00:01<18:36,  1.31it/s]

  0%|          | 3/1464 [00:02<18:34,  1.31it/s]

...

100%|█████████▉| 1463/1464 [18:38<00:00,  1.31it/s]

100%|██████████| 1464/1464 [18:39<00:00,  1.31it/s]

100%|██████████| 1464/1464 [18:39<00:00,  1.31it/s]

100%|██████████| 1/1 [18:56<00:00, 1136.80s/it]

100%|██████████| 1/1 [18:56<00:00, 1136.80s/it]